In [1]:
# import Glop package
from ortools.linear_solver import pywraplp as glp
import lptools as lpt

In [2]:
#Create LP model object
mymodel = glp.Solver('Sports',glp.Solver.GLOP_LINEAR_PROGRAMMING)

In [3]:
inf = mymodel.infinity()

         # name: supply capacity, objective coefficient list
source = {'Muncie': (3 , [8,6]),    #
          'Brazil': (6 , [3,8]),
          'Xenia':  (5,  [9,3])
         }  

        # name: supply capacity, objective coefficient list
transship = {'Louisville': (500, [44,34,34,32]),   #shipping cost per unit to each customer
             'Cincinnati': (500, [57,35,28,24]),   # No supply capacity given
            }   

        # name: demand quantity required
demand = {'Macon': 2, 'Greenwood': 4, 'Concord': 3, 'Chatham': 3}    # weekly customer demand for Just Sports, Sports N Stuff, and Sports Dude

mymodel.Objective().SetMinimization()

In [4]:
# create inbound shipment variables from suppliers to dcs and a dictionary to access them
ship = dict()
for s in source:
    ship[s] = dict()
    (b,coeff_lst) = source[s]
    for (t,c) in zip(transship,coeff_lst):
        ship[s][t] = mymodel.NumVar(0,inf, s + "." + t) 
        mymodel.Objective().SetCoefficient(ship[s][t], c)

In [5]:
# create outbound shipment variables from DCs to Customers and a dictionary to access them
for t in transship:
    ship[t] = dict()
    (b,coeff_lst) = transship[t]
    for (d,c) in zip(demand,coeff_lst):
        ship[t][d] = mymodel.NumVar(0,inf, t + "." + d )
        mymodel.Objective().SetCoefficient(ship[t][d], c)

In [6]:
# create source/supply constraints
for s in source:
    (b,coeff_lst) = source[s]
    constr = mymodel.Constraint(-inf,b,s )
    for t in transship:
        constr.SetCoefficient(ship[s][t], 1)
        
# create DC
for t in transship:
    (b,coeff_lst) = transship[t]
    constr = mymodel.Constraint(-inf,b,t )
    for s in source:
        constr.SetCoefficient(ship[s][t], 1)

In [7]:
lpt.print_model(mymodel)

Variables:
Muncie.Louisville, Muncie.Cincinnati, Brazil.Louisville, Brazil.Cincinnati, Xenia.Louisville, Xenia.Cincinnati, Louisville.Macon, Louisville.Greenwood, Louisville.Concord, Louisville.Chatham, Cincinnati.Macon, Cincinnati.Greenwood, Cincinnati.Concord, Cincinnati.Chatham 

minimize: 8.0*Muncie.Louisville + 6.0*Muncie.Cincinnati + 3.0*Brazil.Louisville + 8.0*Brazil.Cincinnati + 9.0*Xenia.Louisville + 3.0*Xenia.Cincinnati + 44.0*Louisville.Macon + 34.0*Louisville.Greenwood + 34.0*Louisville.Concord + 32.0*Louisville.Chatham + 57.0*Cincinnati.Macon + 35.0*Cincinnati.Greenwood + 28.0*Cincinnati.Concord + 24.0*Cincinnati.Chatham 

Subject To:
Muncie: 1.0*Muncie.Louisville + 1.0*Muncie.Cincinnati <= 3.0
Brazil: 1.0*Brazil.Louisville + 1.0*Brazil.Cincinnati <= 6.0
Xenia: 1.0*Xenia.Louisville + 1.0*Xenia.Cincinnati <= 5.0
Louisville: 1.0*Muncie.Louisville + 1.0*Brazil.Louisville + 1.0*Xenia.Louisville <= 500.0
Cincinnati: 1.0*Muncie.Cincinnati + 1.0*Brazil.Cincinnati + 1.0*Xenia.Cinc

In [8]:
# create demand constraints
for d in demand:
    lb = ub = demand[d]
    constr = mymodel.Constraint(lb,ub,d)
    for t in transship:
        constr.SetCoefficient(ship[t][d], 1)

In [9]:
lpt.print_model(mymodel)

Variables:
Muncie.Louisville, Muncie.Cincinnati, Brazil.Louisville, Brazil.Cincinnati, Xenia.Louisville, Xenia.Cincinnati, Louisville.Macon, Louisville.Greenwood, Louisville.Concord, Louisville.Chatham, Cincinnati.Macon, Cincinnati.Greenwood, Cincinnati.Concord, Cincinnati.Chatham 

minimize: 8.0*Muncie.Louisville + 6.0*Muncie.Cincinnati + 3.0*Brazil.Louisville + 8.0*Brazil.Cincinnati + 9.0*Xenia.Louisville + 3.0*Xenia.Cincinnati + 44.0*Louisville.Macon + 34.0*Louisville.Greenwood + 34.0*Louisville.Concord + 32.0*Louisville.Chatham + 57.0*Cincinnati.Macon + 35.0*Cincinnati.Greenwood + 28.0*Cincinnati.Concord + 24.0*Cincinnati.Chatham 

Subject To:
Muncie: 1.0*Muncie.Louisville + 1.0*Muncie.Cincinnati <= 3.0
Brazil: 1.0*Brazil.Louisville + 1.0*Brazil.Cincinnati <= 6.0
Xenia: 1.0*Xenia.Louisville + 1.0*Xenia.Cincinnati <= 5.0
Louisville: 1.0*Muncie.Louisville + 1.0*Brazil.Louisville + 1.0*Xenia.Louisville <= 500.0
Cincinnati: 1.0*Muncie.Cincinnati + 1.0*Brazil.Cincinnati + 1.0*Xenia.Cinc

In [10]:
# create transshipment flow balance constraints
for t in transship:
    constr = mymodel.Constraint(0,0,t )
    for s in source:
        constr.SetCoefficient(ship[s][t], 1)
    for d in demand:
        constr.SetCoefficient(ship[t][d], -1)

In [11]:
lpt.print_model(mymodel)

Variables:
Muncie.Louisville, Muncie.Cincinnati, Brazil.Louisville, Brazil.Cincinnati, Xenia.Louisville, Xenia.Cincinnati, Louisville.Macon, Louisville.Greenwood, Louisville.Concord, Louisville.Chatham, Cincinnati.Macon, Cincinnati.Greenwood, Cincinnati.Concord, Cincinnati.Chatham 

minimize: 8.0*Muncie.Louisville + 6.0*Muncie.Cincinnati + 3.0*Brazil.Louisville + 8.0*Brazil.Cincinnati + 9.0*Xenia.Louisville + 3.0*Xenia.Cincinnati + 44.0*Louisville.Macon + 34.0*Louisville.Greenwood + 34.0*Louisville.Concord + 32.0*Louisville.Chatham + 57.0*Cincinnati.Macon + 35.0*Cincinnati.Greenwood + 28.0*Cincinnati.Concord + 24.0*Cincinnati.Chatham 

Subject To:
Muncie: 1.0*Muncie.Louisville + 1.0*Muncie.Cincinnati <= 3.0
Brazil: 1.0*Brazil.Louisville + 1.0*Brazil.Cincinnati <= 6.0
Xenia: 1.0*Xenia.Louisville + 1.0*Xenia.Cincinnati <= 5.0
Louisville: 1.0*Muncie.Louisville + 1.0*Brazil.Louisville + 1.0*Xenia.Louisville <= 500.0
Cincinnati: 1.0*Muncie.Cincinnati + 1.0*Brazil.Cincinnati + 1.0*Xenia.Cinc

In [12]:
#solve model and display results
status = mymodel.Solve()
print('Solution Status =',status)
print('Optimal Value = %.2f' % mymodel.Objective().Value())

optimal_value1 = mymodel.Objective().Value()
for v in mymodel.variables():
    print('%s = %.2f' % (v.name(),v.solution_value()))

Solution Status = 0
Optimal Value = 419.00
Muncie.Louisville = 0.00
Muncie.Cincinnati = 1.00
Brazil.Louisville = 6.00
Brazil.Cincinnati = 0.00
Xenia.Louisville = 0.00
Xenia.Cincinnati = 5.00
Louisville.Macon = 2.00
Louisville.Greenwood = 4.00
Louisville.Concord = 0.00
Louisville.Chatham = 0.00
Cincinnati.Macon = 0.00
Cincinnati.Greenwood = 0.00
Cincinnati.Concord = 3.00
Cincinnati.Chatham = 3.00


In [1]:
# only print nonzero shipment variables
print('Variable    LB   Value    UB   Reduced Cost')
for v in mymodel.variables():
    if v.solution_value() != 0:
        print('%8s  %5.1f  %5.1f  %5.1f  %5.2f' % (v.name(),v.lb(),v.solution_value(),v.ub(),v.reduced_cost()))

NameError: name 'mymodel' is not defined

In [14]:
# display all variable information
print('Variable    LB   Value    UB   Reduced Cost')
for v in mymodel.variables():
    print('%8s  %5.1f  %5.1f  %5.1f  %5.2f' % (v.name(),v.lb(),v.solution_value(),v.ub(),v.reduced_cost()))

Variable    LB   Value    UB   Reduced Cost
Muncie.Louisville    0.0    0.0    inf   1.00
Muncie.Cincinnati    0.0    1.0    inf   0.00
Brazil.Louisville    0.0    6.0    inf   0.00
Brazil.Cincinnati    0.0    0.0    inf   6.00
Xenia.Louisville    0.0    0.0    inf   5.00
Xenia.Cincinnati    0.0    5.0    inf   0.00
Louisville.Macon    0.0    2.0    inf   0.00
Louisville.Greenwood    0.0    4.0    inf   0.00
Louisville.Concord    0.0    0.0    inf   7.00
Louisville.Chatham    0.0    0.0    inf   9.00
Cincinnati.Macon    0.0    0.0    inf  12.00
Cincinnati.Greenwood    0.0    0.0    inf   0.00
Cincinnati.Concord    0.0    3.0    inf   0.00
Cincinnati.Chatham    0.0    3.0    inf   0.00


Problem 2

In [20]:
agents = {
    "Martin Products":(1 , [190,175,125,230]),
    "Schmidt Product": (1,[150,235,155,220]),
    "Miller Container": (1, [210,225,135,260]),
    "D&J Burns": (1,[170,185,190,280]),
    "Larbes Furnishings": (1,[220,190,140,240]),
    "Lawler Depot": (1,[270,200,130,260])
}

In [21]:
tasks = {"hub1": 1, "hub2": 1, "hub3":1, "hub4": 1}

In [22]:
#Create LP model object
mymodel = glp.Solver('hubs',glp.Solver.GLOP_LINEAR_PROGRAMMING)
inf = mymodel.infinity()
mymodel.Objective().SetMinimization()

In [23]:
# create all assignment variables and a dictionary of dictionaries to access them
assign =  dict()
for a in agents:
    assign[a] =  dict()
    (b,coeff_lst) =  agents[a]
    for (t,c) in zip(tasks,coeff_lst):
        assign[a][t] =  mymodel.NumVar(0, inf, a + "-" + t)
        mymodel.Objective().SetCoefficient(assign[a][t],c)

In [24]:
# create task constraints
for t in tasks:
    lb = ub = tasks[t]
    
    constr = mymodel.Constraint(lb,ub,t)
    for a in agents:
        constr.SetCoefficient(assign[a][t],1)

In [25]:
# create agent constraints
for a in agents:
    (b,coeff_lst) = agents[a] 
    constr = mymodel.Constraint(-inf , b, a)
    for t in tasks:
        constr.SetCoefficient(assign[a][t], 1)

In [26]:
#solve model and display results
status = mymodel.Solve()
print('Solution Status =',status)
print('Optimal Value = %.2f' % mymodel.Objective().Value())
for v in mymodel.variables():
    print('%s = %.2f' % (v.name(),v.solution_value()))

Solution Status = 0
Optimal Value = 695.00
Martin Products-hub1 = 0.00
Martin Products-hub2 = 1.00
Martin Products-hub3 = 0.00
Martin Products-hub4 = 0.00
Schmidt Product-hub1 = 1.00
Schmidt Product-hub2 = 0.00
Schmidt Product-hub3 = 0.00
Schmidt Product-hub4 = 0.00
Miller Container-hub1 = 0.00
Miller Container-hub2 = 0.00
Miller Container-hub3 = 0.00
Miller Container-hub4 = 0.00
D&J Burns-hub1 = 0.00
D&J Burns-hub2 = 0.00
D&J Burns-hub3 = 0.00
D&J Burns-hub4 = 0.00
Larbes Furnishings-hub1 = 0.00
Larbes Furnishings-hub2 = 0.00
Larbes Furnishings-hub3 = 0.00
Larbes Furnishings-hub4 = 1.00
Lawler Depot-hub1 = 0.00
Lawler Depot-hub2 = 0.00
Lawler Depot-hub3 = 1.00
Lawler Depot-hub4 = 0.00


In [27]:
# only print nonzero assignment variables
print('Variable    LB   Value    UB   Reduced Cost')
for v in mymodel.variables():
    if v.solution_value() != 0:
        print('%8s  %5.1f  %5.1f  %5.1f  %5.2f' % (v.name(),v.lb(),v.solution_value(),v.ub(),v.reduced_cost()))

Variable    LB   Value    UB   Reduced Cost
Martin Products-hub2    0.0    1.0    inf   0.00
Schmidt Product-hub1    0.0    1.0    inf   0.00
Larbes Furnishings-hub4    0.0    1.0    inf   0.00
Lawler Depot-hub3    0.0    1.0    inf   0.00


Problem 3

In [32]:
#Create LP model object
mymodel = glp.Solver('ShortestPath',glp.Solver.GLOP_LINEAR_PROGRAMMING)

In [33]:
inf = mymodel.infinity()

# node dictionary; value = -1 is origin, value = 1 is destination, value = 0 is intermediate node
node = {1:-1,2:0,3:0,4:0,5:0,6:1}

# edge dictionary; (origin,destination):distance/cost
edge = {(1,2):40,
        (1,3):36,
        (2,3):6,
        (2,4):12,
        (2,6):25,
        (3,2):6,
        (3,5):15,
        (4,2):12,
        (4,5):8,
        (4,6):11,
        (5,3):15,
        (5,4):8,
        (5,6):23
       }

mymodel.Objective().SetMinimization()    # minimize total transportation cost

In [34]:
# create a constraint dictionary containing a constraint for each node
constr = dict()
for n in  node:
    b =  node[n]
    constr[n] = mymodel.Constraint(b,b,(str(n)) )

In [35]:
# create a variable dictionary containing a variable for each edge
# add each variable to the objective and its corresponding constraints

path = dict()
for e in edge:
    (o,d) = e
    c = edge[e]
    path[e] = mymodel.NumVar(0,inf, str(o) + '_' + str(d))
    mymodel.Objective().SetCoefficient(path[e],c)
    constr[o].SetCoefficient(path[e],-1)
    constr[d].SetCoefficient(path[e],1)

In [36]:
lpt.print_model(mymodel)

Variables:
1_2, 1_3, 2_3, 2_4, 2_6, 3_2, 3_5, 4_2, 4_5, 4_6, 5_3, 5_4, 5_6 

minimize: 40.0*1_2 + 36.0*1_3 + 6.0*2_3 + 12.0*2_4 + 25.0*2_6 + 6.0*3_2 + 15.0*3_5 + 12.0*4_2 + 8.0*4_5 + 11.0*4_6 + 15.0*5_3 + 8.0*5_4 + 23.0*5_6 

Subject To:
1: - 1.0*1_2 - 1.0*1_3 = -1.0
2: 1.0*1_2 - 1.0*2_3 - 1.0*2_4 - 1.0*2_6 + 1.0*3_2 + 1.0*4_2 = 0.0
3: 1.0*1_3 + 1.0*2_3 - 1.0*3_2 - 1.0*3_5 + 1.0*5_3 = 0.0
4: 1.0*2_4 - 1.0*4_2 - 1.0*4_5 - 1.0*4_6 + 1.0*5_4 = 0.0
5: 1.0*3_5 + 1.0*4_5 - 1.0*5_3 - 1.0*5_4 - 1.0*5_6 = 0.0
6: 1.0*2_6 + 1.0*4_6 + 1.0*5_6 = 1.0

Bounds:
1_2 >= 0.0
1_3 >= 0.0
2_3 >= 0.0
2_4 >= 0.0
2_6 >= 0.0
3_2 >= 0.0
3_5 >= 0.0
4_2 >= 0.0
4_5 >= 0.0
4_6 >= 0.0
5_3 >= 0.0
5_4 >= 0.0
5_6 >= 0.0


In [37]:
#solve model and display results
status = mymodel.Solve()
print('Solution Status =',status)
print('Optimal Value = %.2f' % mymodel.Objective().Value())
for v in mymodel.variables():
    if v.solution_value() != 0:
        print('%s = %.2f' % (v.name(),v.solution_value()))

Solution Status = 0
Optimal Value = 63.00
1_2 = 1.00
2_4 = 1.00
4_6 = 1.00


In [45]:
#Create MILP model object
mymodel = glp.Solver('cars', glp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

In [39]:
inf = mymodel.infinity()

         # name: supply capacity (thousand tons), fixed cost (thousand $), unit shipping cost ($) to each RDC
source = {'Michigan':   (30,320 , [ 2, 6, 10]),   
          'NY1':        (15,175 , [ 3, 4, 9]),   
          'Ohio':       (30,300 , [ 5, 9, 7]),   
          'Mil':        (25,0 , [5, 10, 8]),  
          'California': (25,0 , [5, 10, 8]),  
         }
          
        # name: demand quantity required (thousand tons)
demand = {'Sct': 20, 'Atl': 10, 'Reno': 25,}    # Scranton Atlanta Reno

mymodel.Objective().SetMinimization()    # minimize total plant and transportation cost

In [46]:
inf = mymodel.infinity()

            # name: (type 'c'=continuous or 'i'=integer, lower bound, upper bound, objective coefficient)
variables = {'Michigan':   ('i',0 ,1 , 25),
             'NY1':        ('i',0 ,1 , 35),
             'NY2':        ('i',0 ,1 , 35),
             'Ohio':       ('i',0 ,1 , 40),
             'California': ('i',0 ,1 , 20),
            }

            # name: (lower bound, upper bound, coefficient list)
constraints = {'EBlock': (900 ,inf , [500, 800 ,400 ,900, 200]),
               'Trans':  (900 ,inf , [300, 400, 800, 600, 300]),
              }

mymodel.Objective().SetMinimization()

In [47]:
for v in variables:
    (t,lb,ub,c) = variables[v]
    if t == 'c':
        var = mymodel.NumVar(lb,ub,v)
    elif t == 'i':
        var = mymodel.IntVar(lb,ub,v)
    else:
        print('Invalid variable type =', t)
    mymodel.Objective().SetCoefficient(var,c)

In [48]:
for c in constraints:
    (lb,ub,coeff_lst) = constraints[c]
    constr = mymodel.Constraint(lb,ub,c)
    for (v,coeff) in zip(mymodel.variables(),coeff_lst):
        constr.SetCoefficient(v,coeff)

In [49]:
#solve model and display results
status = mymodel.Solve()
print('Solution Status =',status)
print('Optimal Value = %.2f' % mymodel.Objective().Value())
for v in mymodel.variables():
    print('%s = %.2f' % (v.name(),v.solution_value()))

Solution Status = 0
Optimal Value = 60.00
Michigan = 1.00
NY1 = 0.00
NY2 = 1.00
Ohio = 0.00
California = 0.00
